# Notebook the Generate different word embedding 
Parameter, that can be tested: skipgram size, # of negative samples, dimension of embedding vector

this can be used to test create_word_embeddings.ipynb, after cleaning and vocab-creation

see also create-lemma-embedding-matrix or create-noparty-embedding-matrix

In [ ]:
# imports
import csv
import numpy as np
import pandas as pd
import import_ipynb
import create_skipgrams
import create_word_embeddings
import spacy
from sklearn.model_selection import train_test_split
nlp = spacy.load(
    "de_core_news_lg", exclude=["tok2vec", "ner", "parser", "attribute_ruler"]
)
import gc
from tqdm import tqdm
import tensorflow as tf

In [ ]:
# vocab loading
with open("../vocab/all_vocab_token.csv", mode='r') as infile:
    reader = csv.reader(infile)
    next(reader) #skip header
    vocab = {rows[0]:int(rows[2]) for rows in reader}
vocab['UNK'] = len(vocab) # for new/unknown token
vocab[''] = 0 # padding

In [ ]:
# data collection
data = pd.DataFrame(columns=['text'])

afd = pd.read_csv('../cleaned-data/AfD.csv', sep=",", quoting=csv.QUOTE_NONE, usecols=[2])
cdu = pd.read_csv('../cleaned-data/CDU.csv', sep=",", quoting=csv.QUOTE_NONE, usecols=[2])
csu = pd.read_csv('../cleaned-data/CSU.csv', sep=",", quoting=csv.QUOTE_NONE, usecols=[2])
fdp = pd.read_csv('../cleaned-data/FDP.csv', sep=",", quoting=csv.QUOTE_NONE, usecols=[2])
gru = pd.read_csv('../cleaned-data/GRÜNE.csv', sep=",", quoting=csv.QUOTE_NONE, usecols=[2])
lin = pd.read_csv('../cleaned-data/LINKE.csv', sep=",", quoting=csv.QUOTE_NONE, usecols=[2])
spd = pd.read_csv('../cleaned-data/SPD.csv', sep=",", quoting=csv.QUOTE_NONE, usecols=[2])

data = data.append(afd, ignore_index=True)
data = data.append(cdu, ignore_index=True)
data = data.append(csu, ignore_index=True)
data = data.append(fdp, ignore_index=True)
data = data.append(gru, ignore_index=True)
data = data.append(lin, ignore_index=True)
data = data.append(spd, ignore_index=True)

In [ ]:
# trying to free up ram
del afd
del cdu
del csu
del fdp
del gru
del lin
del spd
gc.collect()

In [ ]:
used_seed = 1234
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
# tokenizing using spacy
data['tokens'] = data['text'].apply([lambda x: [vocab[y.text] if y.text in vocab else vocab['UNK'] for y in nlp(str(x))]])
data_list = data['tokens'].tolist()

In [ ]:
# trying to free up ram
del data
gc.collect()

In [ ]:
# using create_word_embeddings.ipynb to create a word embedding with 3-wide skipgram window, 5 negative samples and 
# a embedding dimension of 300
create_word_embeddings.create_word_embeddings(
    data_list, vocab, [3], [5], [300], "all_data_", used_seed, 15
)